In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
from itertools import combinations
import time
import matplotlib.pyplot as plt

In [96]:
# leer datos de minelib
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
data_name = 'marvin'
pcpsp_path = '../minelib_inputs/' + data_name + '.pcpsp'
prec_path = '../minelib_inputs/' + data_name + '.prec'
upit_path = '../minelib_inputs/'+ data_name + '.upit'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        if linea_lista[0] == 'NAME:':
            dato = linea_lista[1].strip('\n')
            name = dato
        elif linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    lista = linea.split()
                    if lista[2] == 'L':
                        resource_constraint_ub_limits[r,t] = int(lista[-1])
                        resource_constraint_lb_limits[r,t] = '-Infinity' # falta hacer para el caso general
                    else:
                        print('Este problema tiene cotas inferiores.')
                        break
        elif linea_lista[0] == 'OBJECTIVE_FUNCTION:':
            for b in range(nblocks):
                linea = f.readline()
                lista= linea.split()
                for d in range(ndestinations):
                    objective_function_pcpsp[b,d] = float(lista[d+1])
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_COEFFICIENTS:':
                for linea in f:
                    if linea == 'EOF\n':
                        break
                    lista = linea.split()
                    b = int(lista[0])
                    d = int(lista[1])
                    r = int(lista[2])
                    resource_constraint_coefficients[b,r,d] = float(lista[3])

# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

# block value list para upit
bv_list = list() 
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

print('Data base name: %s' % (name))
print('NBLOCKS: %d' % nblocks)
print('NPERIODS: %d' % nperiods)
print('NDESTINATIONS: %d' % ndestinations)
print('NRESOURCE_SIDE_CONSTRAINTS: %d' % nresource_side_constraints)
print('NGENERAL_SIDE_CONSTRAINTS: %d' % ngeneral_side_constraints)
print('DISCOUNT_RATE: %.2f' % discount_rate)

Data base name: marvin
NBLOCKS: 53271
NPERIODS: 20
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.10


In [97]:
# resolver upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(LinExpr([bv_list[i]for i in range(nblocks)], [x[i] for i in range(nblocks)]), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id_upit = [i for i in range(nblocks) if x[i].x==1] # recuperar upit
print('UPIT BLOCKS: %d' % len(blocks_id_upit))

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 1001434.7422
Found heuristic solution: objective 8108630.4111
Found heuristic solution: objective 1.119702e+07
Presolve removed 558793 rows and 44325 columns (presolve time = 5s) ...
Presolve removed 584268 rows and 46077 columns
Presolve time: 7.84s
Presolved: 66363 rows, 7194 columns, 132726 nonzeros
Found heuristic solution: objective 4.772257e+08
Variable types: 0 continuous, 7194 integer (7194 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5285782e+09   0.000000e+00   8.745000e+03      8s
    4851    1.4156554e+09   0.000000e+00   0.000000e+00      8s
    4851    1.4156554e+09   0.000000e+00   0.000000

In [98]:
# funcion de reformulacion
# para PCP_at a PCP_by
def at_by_key(b,d,t):
    if d == 0 and t > 0:
        return b,ndestinations-1,t-1
    elif d > 0:
        return b,d-1,t
    elif d == 0 and t == 0:
        print('at_by_key no esta difinida para los valores d = %d, t = %d' % (d,t))
        
def check_optimality(sol,particion,ell,iteracion):
    for h in range(1,ell+1):
        for (b1,d1,t1),(b2,d2,t2) in combinations(particion[h,iteracion-1],2):
            if not np.isclose(sol[iteracion][b1,d1,t1].x,sol[iteracion][b2,d2,t2].x):
                return False
    return True

def check_opt_eff(sol, particion, ell, iteracion):#  quizas se mejor vectorizar el calculo de las sumas!!!
    for h in range(1,ell[iteracion-1]+1):
        if not (np.isclose(sum([sol[iteracion][b].x for b in particion[h,iteracion-1]]),0) or np.isclose(sum([sol[iteracion][b].x for b in particion[h,iteracion-1]]),len(particion[h,iteracion-1]))):
            return False
    return True

def check_opt_vect(sol, particion, ell, iteracion):
    for h in range(1,ell[iteracion-1]+1):
        arreglo = np.array([sol[iteracion][b].x for b in particion[h,iteracion-1]])
        summ = np.matmul(arreglo,np.ones(arreglo.shape))
        if not (np.isclose(summ,0) or np.isclose(summ,len(particion[h,iteracion-1]))):
            return False
    return True

In [99]:
blocks = blocks_id_upit
blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
resourceTimesPeriod = list(itertools.product(list(range(nresource_side_constraints)),
                                             list(range(nperiods))))
blocks_prime = list(itertools.product(blocks, list(range(ndestinations)),list(range(nperiods))))

In [ ]:
# BZ 

# calculamos la forma equivalente del problema
# PCP_at, llamada PCP_by.
# funcion objetivo
c = {}
for (b,d,t) in blocks_prime:
    c[b,d,t] = (1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d]

# modelo L(PCP_by,mu[k-1])
problema_aux = Model()
# definir variable de PCP_by
x = {}
for (b,d,t) in blocks_prime:
    x[b,d,t] = problema_aux.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "x(%d,%d,%d)" % (b,d,t))
problema_aux.update()
# agregar restricciones de
# precedencias.
# precedencia temporal
for t in range(nperiods-1):
    for b in blocks:
        problema_aux.addConstr(x[b,ndestinations-1,t] <= x[b,0,t+1])

# precedencia en los destinos
for d in range(ndestinations-1):
    for t in range(nperiods):
        for b in blocks:
            problema_aux.addConstr(x[b,d,t] <= x[b,d+1,t])

# precedencia espacial
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        a = int(linea_lista[0])
        if a in blocks:
            for j in range(nvecinos):
                b = int(linea_lista[j+2])
                for t in range(nperiods):
                    problema_aux.addConstr(x[a,ndestinations-1,t] <= x[b,ndestinations-1,t])

# Funcion objetivo de L(PCP_by,mu[k-1])
cx_direct = quicksum([c[b,d,t]*(x[b,d,t]-x[b,d-1,t]) for b,d,t in blocks_prime if d>0 and t>0])\
            +quicksum([c[b,0,t]*(x[b,0,t]-x[b,ndestinations-1,t-1]) for (b,d,t) in blocks_prime if d==0 and t>0])\
            +quicksum([c[b,0,0]*x[b,0,0] for b in blocks])
q = resource_constraint_coefficients
d_rhs = resource_constraint_ub_limits
w = {}
z = {}
eles = {}
eles[0] = 1
k = 1
mu = {}
C = {}
mu[0] = {}
for r,t in resourceTimesPeriod:
    mu[0][r,t] = 0
C[1,0] = set(blocks_prime)
while True:
    # STEP 1: resolver L(PCPby,mu[k-1])
    suma = {}
    side_const = LinExpr()
    LHS = {}
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            sumando_1 = quicksum([q[b,r,d]*(x[b,d,t]-x[b,d-1,t]) for b,d in blockTimesDest if d>0])
            if t > 0:
                LHS[r,t] = sumando_1+quicksum([q[b,r,0]*(x[b,0,t]-x[b,ndestinations-1,t-1]) for b in blocks])
            else:
                LHS[r,0] = sumando_1+quicksum([q[b,r,0]*x[b,0,0] for b in blocks])
    
    fn_objetivo = cx_direct - quicksum([mu[k-1][r,t]*(LHS[r,t]-d_rhs[r,t]) for r,t in resourceTimesPeriod])
    problema_aux.setObjective(fn_objetivo, GRB.MAXIMIZE)
    
    print('\nResolviendo problema auxiliar: k = %d' % k)
    #problema_aux.Params.presolve = 0
    problema_aux.optimize()
    w[k] = x
    # verificar optimalidad de z[k-1]
    init_time = time.time()
    if k>=2 and check_opt_eff(w,C,eles,k):
        print('Algoritmo termino: H^%d w[%d] = 0' % (k-1,k))
        print('Valor optimo de BZ: %.2f' % (model_p2k.ObjVal*(1/obj_scale)))
        break
    print('\nchequear optimalidad tomo: %.2f' % time.time()-init_time)
    print('\nConstruccion del modelo P2^k')
    #STEP 3: encontrar particion de blocks_prime 
    I = [ block for block in blocks_prime if w[k][block].x == 1]
    O = [ block for block in blocks_prime if w[k][block].x == 0]
    count = 0
    for h in range(1,eles[k-1]+1):
        if C[h,k-1].intersection(I):
            count += 1
            C[count,k] = C[h,k-1].intersection(I)
        if C[h,k-1].intersection(O):
            count += 1
            C[count,k] = C[h,k-1].intersection(O)
    
    eles[k] = count
    # STEP 4: resolver P2k
    model_p2k = Model()
    lmbda = {}
    for i in range(1,eles[k]+1):
        lmbda[i] = model_p2k.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name = "lambda%d" % i)

    model_p2k.update()
    # fn objetivo del problema auxiliar P2^k
    x_lmbda = {}
    for b in blocks_prime:
        x_lmbda[b] = quicksum([lmbda[h]*(b in C[h,k])for h in range(1,eles[k]+1)])
    cx_lmbda = quicksum([c[b,d,t]*(x_lmbda[b,d,t]-x_lmbda[b,d-1,t]) for b,d,t in blocks_prime if d>0 and t>0])\
	            +quicksum([c[b,0,t]*(x_lmbda[b,0,t]-x_lmbda[b,ndestinations-1,t-1]) for (b,d,t) in blocks_prime if d==0 and t>0])\
	            +quicksum([c[b,0,0]*x_lmbda[b,0,0] for b in blocks])
    obj_scale = 1e0
    model_p2k.setObjective(obj_scale*cx_lmbda, GRB.MAXIMIZE)
    # agregamos las restricciones
    # para P2k con el cambio de variable
    # agregar restricciones de
    # precedencias.
    for t in range(nperiods-1):
        for b in blocks:
            model_p2k.addConstr(x_lmbda[b,ndestinations-1,t] <= x_lmbda[b,0,t+1])
    
    for d in range(ndestinations-1):
        for t in range(nperiods):
            for b in blocks:
                model_p2k.addConstr(x_lmbda[b,d,t] <= x_lmbda[b,d+1,t])

    with open(prec_path, 'r') as f:
        for linea in f:
            linea_lista = linea.split()
            nvecinos = int(linea_lista[1])
            a = int(linea_lista[0])
            if a in blocks:
                for j in range(nvecinos):
                    b = int(linea_lista[j+2])
                    for t in range(nperiods):
                        model_p2k.addConstr(x_lmbda[a,ndestinations-1,t] <= x_lmbda[b,ndestinations-1,t])
    
    # agregar side constraints: si at_by_key:
    LHS_lmbda = {}
    #for r in range(nresource_side_constraints):
    #    for t in range(nperiods):
    #        LHS_lmbda[r,t] = quicksum([q[b,r,d]*(quicksum([lmbda[j]*((b,d,t) in C[j,k]) for j in range(1,eles[k]+1)])-quicksum([lmbda[j]*((b,d-1,t) in C[j,k]) for j in range(1,eles[k]+1)])) for (b,d) in blockTimesDest])
    #        if t>0:
    #            LHS_lmbda[r,t] = LHS_lmbda[r,t] + quicksum([q[b,r,0]*(quicksum([lmbda[j]*((b,0,t) in C[j,k]) for j in range(1,eles[k]+1)])-quicksum([lmbda[j]*((b,ndestinations-1,t-1) in C[j,k]) for j in range(1,eles[k]+1)])) for b in blocks])
    #        else:
    #            LHS_lmbda[r,0] = LHS_lmbda[r,0] + quicksum([q[b,r,0]*(quicksum([lmbda[j]*((b,0,0) in C[j,k]) for j in range(1,eles[k]+1)])) for b in blocks])
    
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            sumando_1 = quicksum([q[b,r,d]*(x_lmbda[b,d,t]-x_lmbda[b,d-1,t]) for b,d in blockTimesDest if d>0])
            if t > 0:
                LHS_lmbda[r,t] = sumando_1+quicksum([q[b,r,0]*(x_lmbda[b,0,t]-x_lmbda[b,ndestinations-1,t-1]) for b in blocks])
            else:
                LHS_lmbda[r,0] = sumando_1+quicksum([q[b,r,0]*x_lmbda[b,0,0] for b in blocks])
    
    # agregar side constraints Dx <= d
    side_const = {}
    sc_scale = 1e0 # ponderador para side constraints (Dx <=d)
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            side_const[r,t] = model_p2k.addConstr(sc_scale*LHS_lmbda[r,t] <= sc_scale*d_rhs[r,t], name='side_const[%d,%d]' % (r,t))
        
    print('\nResolviendo master problem: k=%d' % k)
    #model_p2k.Params.presolve = 0
    model_p2k.optimize()
    #break # para chequear si tuvo warnings en p2^k
    # recuperar variables duales mu[k]
    mu[k] = {}
    for r in range(nresource_side_constraints):
        for t in range(nperiods):
            mu[k][r,t] = side_const[r,t].pi
    if all([np.isclose(mu[k][r,t], mu[k-1][r,t]) for r,t in resourceTimesPeriod]): # Cambiar por una tolerancia!!!
        # recuperar z[k] la solucion del optimo
        print('Algoritmo termino: mu[%d] = mu[%d]' % (k,k-1))
        print('Valor optimo de BZ: %.2f' % (model_p2k.ObjVal*(1/obj_scale)))
        break
    k += 1


Resolviendo problema auxiliar: k = 1
Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 91883 rows and 193842 columns (presolve time = 5s) ...
Presolve removed 92798 rows and 215021 columns
Presolve time: 8.04s
Presolved: 247842 rows, 2175823 columns, 4102264 nonzeros

Ordering time: 0.00s

Barrier performed 0 iterations in 136.40 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 374520 iterations and 136.48 seconds
Optimal objective  1.441393765e+09

Resolviendo master problem: k=1
Optimize a model with 2050244 rows, 2 columns and 54910 nonzeros
Coefficient statistics:
  Matrix range     [2e-11, 3e+08]
  Objective range  [1e+09, 2e+09]
  Bounds range     

  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0119646e+40   6.412338e+36   1.011965e+10      1s
    8208    1.5753411e+09   1.818830e+05   0.000000e+00      7s
   18649    1.1952364e+09   2.627700e+04   0.000000e+00     17s
   19762    1.1944161e+09   0.000000e+00   0.000000e+00     19s

Solved in 19762 iterations and 19.30 seconds
Optimal objective  1.194416089e+09

Resolviendo master problem: k=6
Optimize a model with 2050244 rows, 9 columns and 193242 nonzeros
Coefficient statistics:
  Matrix range     [3e-11, 5e+08]
  Objective range  [3e+05, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2050220 rows and 0 columns
Presolve time: 0.30s
Presolved: 24 rows, 10 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3388648e+0

  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2050204 rows and 0 columns
Presolve time: 0.30s
Presolved: 40 rows, 14 columns, 91 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3388648e+09   1.034790e+03   0.000000e+00      0s
      20    5.8809217e+08   0.000000e+00   0.000000e+00      1s

Solved in 20 iterations and 0.84 seconds
Optimal objective  5.880921740e+08

Resolviendo problema auxiliar: k = 12
Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 9e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5360479e+39   3.766883e+36   5.536048e+09      1s
   10571    1.4619762e+09   1.453410e+05   0.0

Coefficient statistics:
  Matrix range     [1e-11, 5e+08]
  Objective range  [3e+05, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2050195 rows and 0 columns
Presolve time: 0.31s
Presolved: 49 rows, 17 columns, 109 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3388648e+09   1.034790e+03   0.000000e+00      0s
      23    6.7598303e+08   0.000000e+00   0.000000e+00      1s

Solved in 23 iterations and 0.91 seconds
Optimal objective  6.759830264e+08

Resolviendo problema auxiliar: k = 15
Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 9e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.656


Solved in 107844 iterations and 73.11 seconds
Optimal objective  8.856693922e+08

Resolviendo master problem: k=17
Optimize a model with 2050244 rows, 101 columns and 1364122 nonzeros
Coefficient statistics:
  Matrix range     [3e-11, 3e+08]
  Objective range  [2e+03, 2e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+07, 6e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2049859 rows and 0 columns
Presolve time: 1.14s
Presolved: 385 rows, 101 columns, 1104 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4414025e+09   5.045557e+03   0.000000e+00      1s
     155    8.2029072e+08   0.000000e+00   0.000000e+00      2s

Solved in 155 iterations and 2.03 seconds
Optimal objective  8.202907239e+08

Resolviendo problema auxiliar: k = 18
Optimize a model with 2050204 rows, 340640 columns and 4100408 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1

  Objective range  [2e-03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5615385e+38   3.158693e+36   6.561538e+08      1s
    7036    8.8885713e+08   1.004150e+05   0.000000e+00      9s
   14582    8.5637489e+08   2.448400e+04   0.000000e+00     18s
   17155    8.5396001e+08   1.177500e+04   0.000000e+00     22s
   18600    8.5301334e+08   3.593000e+03   0.000000e+00     27s
   19293    8.5270995e+08   2.140000e+03   0.000000e+00     31s
   19986    8.5254482e+08   6.090000e+02   0.000000e+00     35s
   20679    8.5242532e+08   1.117000e+03   0.000000e+00     40s
   21372    8.5236363e+08   1.852500e+04   0.000000e+00     49s
   22065    8.5226916e+08   1.451400e+04   0.000000e+00     57s
   22758    8.5213067e+08   4.883000e+03   0.000000e+00     65s
   23642    8.5194506e+08   0.000000e+00   0.000000e+00     72s

Solved in 23642 iterations and 71.94 seconds
Optimal objective  8

  Objective range  [2e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7885793e+37   1.456677e+37   3.788579e+07      1s
    5063    8.4606733e+08   4.528300e+04   0.000000e+00     13s
    7205    8.4478065e+08   5.436000e+03   0.000000e+00     16s
    7898    8.4474026e+08   5.180000e+02   0.000000e+00     22s
    8367    8.4471812e+08   0.000000e+00   0.000000e+00     26s

Solved in 8367 iterations and 26.19 seconds
Optimal objective  8.447181237e+08


In [94]:
eles[7]

13

In [35]:
len(O)+len(I)#STEP 3: encontrar particion de blocks_prime (H^k w[k]=h^k)
I = [ block for block in blocks_prime if w[k][block].x == 1]
O = [ block for block in blocks_prime if w[k][block].x == 0]
count = 0
for h in range(1,eles[k-1]+1):
    if C[h,k-1].intersection(I):
        count += 1
        C[count,k] = C[h,k-1].intersection(I)
    if C[h,k-1].intersection(O):
        count += 1
        C[count,k] = C[h,k-1].intersection(O)
            
l = count

In [29]:
I = [ block for block in blocks_prime if w[k][block].x == 1]
O = [ block for block in blocks_prime if w[k][block].x == 0]
len(C[4,k-1].intersection(O))

86

In [44]:
eles[k-1]

3

In [84]:
for h in range(1,eles[k-1]+1):
    print('C[%d,%d] tiene size: %d' % (h,k-1,len(C[h,k-1])))
for h in range(1,eles[k-1]+1):
    print('Para C[%d,%d] los ws[%d] suman: %f' % (h,k-1,k,sum([w[k][b].x for b in C[h,k-1]])))

C[1,7] tiene size: 9041
C[2,7] tiene size: 234
C[3,7] tiene size: 990
C[4,7] tiene size: 50
C[5,7] tiene size: 24
C[6,7] tiene size: 50
C[7,7] tiene size: 24
C[8,7] tiene size: 86
C[9,7] tiene size: 12
C[10,7] tiene size: 8
C[11,7] tiene size: 46
C[12,7] tiene size: 2
C[13,7] tiene size: 2141
Para C[1,7] los ws[8] suman: 9041.000000
Para C[2,7] los ws[8] suman: 234.000000
Para C[3,7] los ws[8] suman: 990.000000
Para C[4,7] los ws[8] suman: 50.000000
Para C[5,7] los ws[8] suman: 24.000000
Para C[6,7] los ws[8] suman: 50.000000
Para C[7,7] los ws[8] suman: 0.000000
Para C[8,7] los ws[8] suman: 0.000000
Para C[9,7] los ws[8] suman: 12.000000
Para C[10,7] los ws[8] suman: 0.000000
Para C[11,7] los ws[8] suman: 0.000000
Para C[12,7] los ws[8] suman: 2.000000
Para C[13,7] los ws[8] suman: 0.000000


In [85]:
len(blocks_prime)

12708

In [87]:
sum([len(C[h,k-1]) for h in range(1,eles[k-1]+1)])

12708

In [64]:
ell = eles[4]
count = 1
for h in range(1,ell+1):
    d = 0 
    if C[h,3].intersection(I):
        C[count,4] = C[h,3].intersection(I)
        d = 1
    if C[h,3].intersection(O):
        if d == 0:
            C[count,4] = C[h,3].intersection(O)
            count += 1
        else:
            C[count+1,4] = C[h,3].intersection(O)
            count += 2
ell_dps = count-1

In [67]:
eles[4]

6

In [71]:
len(C[6,4])

2141

In [27]:
k

5

In [39]:
var_back = {}
for (b,d,t) in blocks_prime:
    var_back[b,d,t] = sum(lmbda[j].x*((b,d,t) in C[j,k-1]) for j in range(1,l+1))
opt = sum([c[b,d,t]*(var_back[b,d,t]-var_back[b,d-1,t]) for b,d,t in blocks_prime if d>0 and t>0])\
            +sum([c[b,0,t]*(var_back[b,0,t]-var_back[b,ndestinations-1,t-1]) for (b,d,t) in blocks_prime if d==0 and t>0])\
            +sum([c[b,0,0]*var_back[b,0,0] for b in blocks])
print('OPT de BZ: %.2f' % opt)

OPT de BZ: 8663560.30


In [ ]:
mu[1]

In [12]:
valor = quicksum([c[b,d,t]*(x_lmbda[b,d,t]-x_lmbda[b,d-1,t]) for b,d,t in blocks_prime if d>0 and t>0])\
	            +quicksum([c[b,0,t]*(x_lmbda[b,0,t]-x_lmbda[b,ndestinations-1,t-1]) for (b,d,t) in blocks_prime if d==0 and t>0])\
	            +quicksum([c[b,0,0]*x_lmbda[b,0,0] for b in blocks])


<gurobi.LinExpr: -33773.67209259259 lambda1 + -0.0 lambda2 + 33773.67209259259 lambda1 + 0.0 lambda2 + -31271.918604252398 lambda1 + -0.0 lambda2 + 31271.918604252398 lambda1 + 0.0 lambda2 + -28955.480189122583 lambda1 + -0.0 lambda2 + 28955.480189122583 lambda1 + 0.0 lambda2 + -26810.62980474313 lambda1 + -0.0 lambda2 + 26810.62980474313 lambda1 + 0.0 lambda2 + -24824.65722661401 lambda1 + -0.0 lambda2 + 24824.65722661401 lambda1 + 0.0 lambda2 + 22989.92222222222 lambda1 + 0.0 lambda2 + -0.0 lambda1 + -22989.92222222222 lambda2 + 21286.965020576128 lambda1 + 0.0 lambda2 + -21286.965020576128 lambda1 + -0.0 lambda2 + 19710.152796829745 lambda1 + 0.0 lambda2 + -19710.152796829745 lambda1 + -0.0 lambda2 + 18250.14147854606 lambda1 + 0.0 lambda2 + -18250.14147854606 lambda1 + -0.0 lambda2 + 16898.279146801906 lambda1 + 0.0 lambda2 + -16898.279146801906 lambda1 + -0.0 lambda2 + 31802.975 lambda1 + 0.0 lambda2 + -0.0 lambda1 + -31802.975 lambda2 + 29447.199074074073 lambda1 + 0.0 lambda2 + 

In [33]:
z[k] = {}
for (b,d,t) in blocks_prime:
	    	z[k][b,d,t] = sum(lmbda[j].x*((b,d,t) in C[j,k-1]) for j in range(1,l+1))

In [14]:
t0 = time.time()
for h in range(1,l+1):
    for (b1,d1,t1) in C[h,k-1]:
        for (b2,d2,t2) in C[h,k-1]:
            pass
print(time.time()-t0)

8.293293237686157


In [15]:
t0 = time.time()
for h in range(1,l+1):
    for (b1,d1,t1),(b2,d2,t2) in combinations(C[h,k-1],2):
        pass
print(time.time()-t0)

8.261805057525635


In [17]:
for tupla in combinations(set([2,1,4,5,67,7]),2):
    print(tupla)

(1, 2)
(1, 67)
(1, 4)
(1, 5)
(1, 7)
(2, 67)
(2, 4)
(2, 5)
(2, 7)
(67, 4)
(67, 5)
(67, 7)
(4, 5)
(4, 7)
(5, 7)


In [8]:
z[k] = {}
for (b,d,t) in blocks_prime:
    z[k][b,d,t] = sum(lmbda[j].x*((b,d,t) in C[j,k-1]) for j in range(1,l+1))
    
z[k].values

<function dict.values>

In [12]:
model_p2k.ObjVal*(1/obj_scale)

8663560.298788792

In [46]:
# calculo alternativo de las expresiones
# para ver si mejora el escalamiento numerico
cx_direct = quicksum([c[b,d,t]*(x[b,d,t]-x[b,d-1,t]) for b,d,t in blocks_prime if d>0 and t>0])\
            +quicksum([c[b,0,t]*(x[b,0,t]-x[b,ndestinations-1,t-1]) for (b,d,t) in blocks_prime if d==0 and t>0])\
            +quicksum([c[b,0,0]*x[b,0,0] for b in blocks])
LHS = {}
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        sumando_1 = quicksum([q[b,r,d]*(x[b,d,t]-x[b,d-1,t]) for b,d in blockTimesDest if d>0])
        if t > 0:
            LHS[r,t] = sumando_1+quicksum([q[b,r,0]*(x[b,0,t]-x[b,ndestinations-1,t-1]) for b in blocks])
        else:
            LHS[r,0] = sumando_1+quicksum([q[b,r,0]*x[b,0,0] for b in blocks])

In [28]:
import math

True

In [24]:
max(q.values())

5664.0